In [1]:
import json
import sys
from IPython.display import display, Markdown
sys.path.append("../src")
from agentix import AgentixConfig
from agentix.agent import agentix
from agentix.sessions import assemble_prompts

In [2]:
test_project_path = "/home/mpeters/starbucks/projects/retail-fraud-analytics/python"

In [ ]:
config=AgentixConfig()
config.model="gpt"
config.system=["prompt_classification"]
config.tools=["cst"]
config.with_frontend=True
config.debug=False
config.user = [
    ("Create a plan to identify classes that implement both modif() and requires() "
    "methods so you can create a class decortator to describe the modifycation.  Identify tools to call"),
]


In [3]:
config=AgentixConfig()
config.model="gpt"
config.system=["python_coder", "modifier_class_decorator","structured_response"]
config.tools=["cst"]
config.with_frontend=True
config.debug=False
config.user = [
    ("Create a plan to identify classes that implement both modif() and requires() "
    "methods so you can create a class decortator to describe the modifycation.  Identify tools to call"),
]


In [4]:
p = assemble_prompts(config, [], 4000)
print(json.dumps(p, indent=2).encode("utf8").decode("unicode_escape"))

{
  "model": "gpt",
  "messages": [
    {
      "role": "tool_calls",
      "content": "[TOOLS]
[
  [
    {
      "type": "function",
      "function": {
        "name": "module_files",
        "description": "Get all Python files in a module directory (recursive) whose path (relative",
        "parameters": {
          "properties": {
            "root_path": {
              "type": "str"
            },
            "module_prefix": {
              "type": "str"
            }
          }
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "cst_tree",
        "description": "Parse a Python file into a LibCST Module (Concrete Syntax Tree).",
        "parameters": {
          "properties": {
            "file_path": {
              "type": "str"
            }
          }
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "cst_modules",
        "description": "Parse multiple Python files into LibCST Modules.

In [5]:
results:dict = agentix(config)

Debug: args.session = agentix_session
Debug: Invoking manage_sessions
Managing session: agentix_session
Creating new session...
Debug: Calling summarize_user_prompt with args: AgentixConfig(debug=False, list_models=False, list_sessions=False, list_prompts=False, session='agentix_session', system=['python_coder', 'modifier_class_decorator', 'structured_response'], model='gpt-oss:latest', temperature=0.7, user=['Create a plan to identify classes that implement both modif() and requires() methods so you can create a class decortator to describe the modifycation.  Identify tools to call'], file_path=None, replace_file=False, serve=False, port=8000, with_frontend=True, tools=['cst'])
Session ClassModifierInspector_Plan_Tools_Identification created.
Debug: args.session = ClassModifierInspector_Plan_Tools_Identification
Debug: args = AgentixConfig(debug=False, list_models=False, list_sessions=False, list_prompts=False, session='ClassModifierInspector_Plan_Tools_Identification', system=['pytho

In [16]:
for r in results:
    for k,v in r.items():
        match k:
            case "attachment":
                match type(v).__name__:
                    case "dict":
                        if "name" in v:
                            display(Markdown(f"### Attachment: `\n"))
                            display(Markdown("-----"))
                            data = v.get("data","")
                            language = v.get("language","text")
                            display(Markdown(f"### Attachment Content:\n```{data}\n```\n"))
            case "markdown":
                display(Markdown(v))
            case "next_steps":
                display(Markdown("### Next Steps\n"))
                for step in v:
                    display(f"-- {step}")
            case _:
                print(k)
                print(v)
                print("-----")


agent
Here is a concise plan and the tools you can use to identify classes that implement both `modify` and `requires` methods.
-----


### Attachment: `


-----

### Attachment Content:
```# Plan to Find Classes with `modify` and `requires`

## 1. Discover Python files
- Use `find` or `ripgrep` to recursively list all `.py` files in the target repository.
  ```bash
  find . -name "*.py"
  # or
  rg --files --type python
  ```

## 2. Static AST inspection
- Write a small Python script that:
  1. Iterates over each file.
  2. Parses the file with `ast.parse()`.
  3. Walks the AST looking for `ast.ClassDef` nodes.
  4. For each class, checks for:
     - A method named `modify` (or `modif` if that was a typo).
     - A method or property named `requires`.
- Store matches in a list of dictionaries with keys: `module`, `class_name`, `file_path`.

```python
import ast, pathlib

matches = []
for path in pathlib.Path('.').rglob('*.py'):
    tree = ast.parse(path.read_text(), filename=str(path))
    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            has_modify = any(
                isinstance(m, ast.FunctionDef) and m.name == 'modify'
                for m in node.body
            )
            has_requires = any(
                (isinstance(m, ast.FunctionDef) and m.name == 'requires') or
                (isinstance(m, ast.Attribute) and m.attr == 'requires')
                for m in node.body
            )
            if has_modify and has_requires:
                matches.append({
                    'module': path.parent.as_posix(),
                    'class': node.name,
                    'path': str(path)
                })
print(matches)
```

## 3. Optional: Use Existing Linters / Static Analyzers
- **Pylint**: Configure a custom checker or use its *generated* AST to filter classes.
- **Ruff**: Fast linting; can be scripted to emit diagnostics for missing methods.
- **Mypy**: Type checking can reveal classes lacking required signatures.
- **Jedi** or **Pyright**: For dynamic analysis if the code uses complex metaprogramming.

## 4. Automate Decorator Insertion
- Once you have the list of target classes, generate decorator stubs.
- Use `ast.NodeTransformer` to add `@etl_step(...)` decorator as described in the prompt.
- Write back the modified AST to file with `ast.unparse()` (Python 3.9+) or `astor`.

## 5. Tool Summary
| Tool | Purpose |
|------|---------|
| `find` / `rg` | Locate all Python source files |
| `ast` (Python stdlib) | Parse and analyze class/method structure |
| `pylint` | Optional linting and custom rule creation |
| `ruff` | Fast linting, can be scripted |
| `mypy` | Type‑based static analysis |
| `jedi` / `pyright` | Dynamic/static code navigation |
| `astor` / `black` | Pretty‑print and format modified AST |

## Next Steps
- Run the discovery script on a fresh clone of the repo.
- Validate the output list manually.
- Proceed to write the decorator insertion logic.

```


In [17]:
# config=AgentixConfig()
# config.system=["python_coder", "modifier_class_decorator","structured_response"]
# config.file = test_project_path
# config.user = ("Create a plan to identify classes that implement both modif() and requires() "
#               "methods so you can create a class decortator to describe the modifycation.")

In [ ]:
# agentix(config)